In [5]:
#pip install folium
import requests
import pandas as pd
import folium


In [6]:
zipcodes = [19701, 19702, 19703, 19706, 19707, 19708, 19709, 19710, 19711, 19712, 19713, 19714, 19715, 19716, 19717, 19718, 19720, 19721, 19725, 19726, 19730, 19731, 19732, 19733, 19734, 19735, 19736, 19801, 19802, 19803, 19804, 19805, 19806, 19807, 19808, 19809, 19810, 19850, 19880, 19884, 19885, 19886, 19887, 19889, 19890, 19891, 19892, 19893, 19894, 19895, 19896, 19897, 19898, 19899, 19901, 19902,19903, 19904, 19905, 19906,19930, 19931, 19933, 19934, 19936, 19938, 19939, 19940, 19941, 19943, 19944, 19945, 19946, 19947, 19950, 19951, 19952, 19953, 19954, 19955, 19956, 19958, 19960, 19961, 19962, 19963, 19964, 19966, 19967, 19968, 19969, 19970, 19971, 19973, 19975, 19977, 19979, 19980]


In [76]:

li=[]
no_data_to_show = []
for zipcode in zipcodes:
  url = 'https://myhealthycommunity.dhss.delaware.gov/locations/zip-code-'+str(zipcode)+'/download_covid_19_data/cases'
  try:
      df = pd.read_csv(url)
      li.append(df)
      
  except pd.errors.ParserError:
      no_data_to_show.append(zipcode)
  
  df = pd.concat(li, axis=0, ignore_index=True)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13651 entries, 0 to 13650
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Location      13651 non-null  object 
 1   County        13651 non-null  object 
 2   Statistic     13651 non-null  object 
 3   Value         13651 non-null  float64
 4   Year          13651 non-null  object 
 5   Month         13651 non-null  object 
 6   Day           13651 non-null  object 
 7   Date used     13651 non-null  object 
 8   Unit          13651 non-null  object 
 9   Age adjusted  13651 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.0+ MB


In [8]:
#zipcode populations

zip_pop ={19720:59664, 19702:55537, 19711:52547, 19709:42480, 19701:40173, 19808:38657, 19805:38457, 19904:37467, 19901:37182, 19713:30260, 19966:29324, 19958:26235, 19810:25978, 19802:25715, 19977:25670, 19973:24745, 19803:21867, 19963:20743, 19947:19992, 19804:17511, 19801:17035, 19707:15821, 19703:15362, 19809:15139, 19956:15117, 19934:13748, 19971:13739, 19734:13115, 19943:12887, 19962:11710, 19968:11659, 19933:10903, 19952:10094, 19806:9887, 19975:9661, 19938:8856, 19970:8505, 19807:7515, 19950:7453, 19939:7399, 19945:6771, 19960:6505, 19940:5575, 19946:5294, 19953:4545, 19717:3868, 19941:3270, 19930:2300, 19951:2103, 19706:1803, 19954:1523, 19716:1429, 19967:1246, 21912:1139, 19964:1136, 19730:1100, 19979:719, 19944:697, 19731:322, 19936:319, 19931:309, 19902:300, 19733:89, 19732:88, 19736:66, 19955:53, 19735:9}

In [9]:
#convert location to int zipcodes and rename column
df['Location'] = df['Location'].apply(lambda x: int(str(x[9::])))
df.rename(columns = {'Location':'Zipcode'}, inplace = True) 
#add population column
df['Zipcode Population'] = df['Zipcode'].map(zip_pop)


In [10]:
#add new date column and drop Month, Day and Year Columns 
df.replace({'Month': {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6:'06', 7:'07', 8:'08',  9:'09'},'Day': {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6:'06', 7:'07', 8:'08', 9:'09'}}, inplace=True)

df['Date'] = df['Year'].apply(str) + df['Month'].apply(str) + df['Day'].apply(str)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

df = df.drop(['Day','Month','Year'], axis=1)


In [11]:
#add zipcode date which will serve as the primary key}
df['Zipcode Date'] = df['Zipcode'].astype(str) + " " + df['Date'].astype(str)

In [31]:
#latitudes and longitudes by zipcode
lats_longs = {'19725':[39.564499,-75.597047],'19884':[39.564499,-75.597047],'19980':[39.07027,-75.57057],'19902':[39.131958,-75.46953], '19947':[38.676552,-75.39269],'19708':[39.577567,-75.690069],'19733':[39.555794,-75.65058], '19894': [39.564499,-75.597047], '19898':[39.564499,-75.597047],'19971':[38.711512,-75.09677],'19706':[39.573744,-75.59204],'19951':[38.682345,-75.23286],'19901':[39.16426,-75.51163],'19889':[39.564499,-75.597047],'19966':[38.601355,-75.2411],'19903':[39.10868,-75.448023],'19808':[39.734279,-75.6631],'19930':[38.536354,-75.06062],'19711':[39.700561,-75.7431],'19721':[39.564499,-75.597047],'19810':[39.817645,-75.50242],'19904':[39.161639,-75.5587],'19975':[38.463751,-75.15642],'19886':[39.564499,-75.597047],'19766':[39.567092,-75.601491],'19961':[39.166323,-75.448289],'19700':[39.570322,-75.590662],'19880':[39.564499,-75.597047],'19939':[38.557501,-75.21465],'19934':[39.094699,-75.58871],'19734':[39.386601,-75.66801],'19999':[38.542771,-75.05666],'19950':[38.818541,-75.60966],'19931':[38.570238,-75.6147],'19726':[39.564499,-75.597047],'19940':[38.469655,-75.5669], '19943':[39.011387,-75.58978],'19809':[39.771663,-75.49656],'19977':[39.29799,-75.59391],'19893':[39.564499,-75.597047],'19807':[39.787512,-75.60256],'19967':[38.54597,-75.11175],'19885':[39.564499,-75.597047],'19709':[39.479602,-75.6932],'19736':[39.790911,-75.64934],'19703':[39.800945,-75.46455],'19897':[39.564499,-75.597047],'19718':[39.564499,-75.597047],'19979':[39.0461,-75.57185],'19717':[39.564499,-75.597047],'19701':[39.598203,-75.69945],'19806':[39.758563,-75.56413],'19968':[38.772648,-75.28665],'19891':[39.564499,-75.597047],'19814':[39.713625,-75.596283],'19964':[39.098772,-75.73943],'19970':[38.55044,-75.09928],'19944':[38.459314,-75.05356],'19938':[39.265066,-75.6807],'19973':[38.643248,-75.61102],'19956':[38.549721,-75.55304],'19899':[39.734792,-75.624617],'19803':[39.793962,-75.53401],'19936':[39.218448,-75.584848], '19887': [39.564499,-75.597047],'19801':[39.738563,-75.54833],'19710':[39.788562,-75.58881],'19732':[39.794496,-75.57433],'19731':[39.518164,-75.57656],'19960':[38.855872,-75.39889],'19707':[39.784014,-75.68586],'19952':[38.916908,-75.61343],'19735':[39.794396,-75.597598],'19804':[39.721062,-75.60806],'19890':[39.564499,-75.597047],'19933':[38.73635,-75.60807],'19942':[38.868387,-75.57907],'19963':[38.922806,-75.41449], '19892':[39.564499,-75.597047],'19730':[39.456484,-75.65976],'19802':[39.756213,-75.53312],'19946':[39.037803,-75.46634],'19962': [39.06517,-75.49858], '19958':[38.746207,-75.16282], '19955':[39.227313,-75.66549],'19714':[39.564499,-75.597047],'19895': [39.564499,-75.597047], '19702':[39.626297,-75.71386],'19712':[39.564499,-75.597047], '19969':[38.725562,-75.353387],'19954':[38.909621,-75.51264], '19716':[39.564499,-75.597047], '19805':[39.745377,-75.58251],'19715':[39.564499,-75.597047],'19941':[38.80136,-75.42595], '19850':[39.564499,-75.597047], '19720':[39.669219,-75.59003],'19896':[39.564499,-75.597047],'19945':[38.511469,-75.18364], '19713':[39.669211,-75.71796], '19905':[39.10868,-75.448023],'19953':[39.150822,-75.70428]}


In [68]:
long_lagsdf = pd.DataFrame.from_dict(lats_longs, orient='index', columns=['Zipcode Latitude', 'Zipcode longitude'])

In [75]:
pd.DataFrame.to_dict(long_lagsdf, 'index')

{'19725': {'Zipcode Latitude': 39.564499, 'Zipcode longitude': -75.597047},
 '19884': {'Zipcode Latitude': 39.564499, 'Zipcode longitude': -75.597047},
 '19980': {'Zipcode Latitude': 39.07027, 'Zipcode longitude': -75.57057},
 '19902': {'Zipcode Latitude': 39.131958, 'Zipcode longitude': -75.46953},
 '19947': {'Zipcode Latitude': 38.676552, 'Zipcode longitude': -75.39269},
 '19708': {'Zipcode Latitude': 39.577567, 'Zipcode longitude': -75.690069},
 '19733': {'Zipcode Latitude': 39.555794, 'Zipcode longitude': -75.65058},
 '19894': {'Zipcode Latitude': 39.564499, 'Zipcode longitude': -75.597047},
 '19898': {'Zipcode Latitude': 39.564499, 'Zipcode longitude': -75.597047},
 '19971': {'Zipcode Latitude': 38.711512, 'Zipcode longitude': -75.09677},
 '19706': {'Zipcode Latitude': 39.573744, 'Zipcode longitude': -75.59204},
 '19951': {'Zipcode Latitude': 38.682345, 'Zipcode longitude': -75.23286},
 '19901': {'Zipcode Latitude': 39.16426, 'Zipcode longitude': -75.51163},
 '19889': {'Zipcode La

In [32]:
lats = dict()
longs = dict()
for key, value in lats_longs.items():
  lats[int(key)]= float(value[0])
  longs[int(key)]=float(value[1])  

In [34]:
df['Zipcode latitude'] = df['Zipcode'].map(lats)
df['Zipcode longitude'] = df['Zipcode'].map(longs)


,Zipcode,County,Statistic,Value,Date used,Unit,Age adjusted,Zipcode Population,Date,Zipcode Date,Zipcode latitude,Zipcode longitude
0,19701,New Castle County,Cumulative Number of Positive Cases,0.0,Date reported to state,"rate per 10,000 people",False,40173,2020-03-08,19701 2020-03-08,39.598203,-75.69945
1,19701,New Castle County,Cumulative Number of Positive Cases,0.0,Date reported to state,"rate per 10,000 people",False,40173,2020-03-09,19701 2020-03-09,39.598203,-75.69945
2,19701,New Castle County,Cumulative Number of Positive Cases,0.0,Date reported to state,"rate per 10,000 people",False,40173,2020-03-10,19701 2020-03-10,39.598203,-75.69945
3,19701,New Castle County,Cumulative Number of Positive Cases,0.0,Date reported to state,"rate per 10,000 people",False,40173,2020-03-11,19701 2020-03-11,39.598203,-75.69945
4,19701,New Castle County,Cumulative Number of Positive Cases,0.0,Date reported to state,"rate per 10,000 people",False,40173,2020-03-12,19701 2020-03-12,39.598203,-75.69945
...,...,...,...,...,...,...,...,...,...,...,...,...
13551,19977,Kent County,Cumulative Number of Positive Cases,727.1,Date reported to state,"rate per 10,000 people",False,25670,2021-01-01,19977 2021-01-01,39.297990,-75.59391
13552,19977,Kent County,Cumulative Number of Positive Cases,734.1,Date reported to state,"rate per 10,000 people",False,25670,2021-01-02,19977 2021-01-02,39.297990,-75.59391
13553,19977,Kent County,Cumulative Number of Positive Cases,745.5,Date reported to state,"rate per 10,000 people",False,25670,2021-01-03,19977 2021-01-03,39.297990,-75.59391
13554,19977,Kent County,Cumulative Number of Positive Cases,757.6,Date reported to state,"rate per 10,000 people",False,25670,2021-01-04,19977 2021-01-04,39.297990,-75.59391


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13556 entries, 0 to 13555
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Zipcode             13556 non-null  int64         
 1   County              13556 non-null  object        
 2   Statistic           13556 non-null  object        
 3   Value               13556 non-null  float64       
 4   Date used           13556 non-null  object        
 5   Unit                13556 non-null  object        
 6   Age adjusted        13556 non-null  object        
 7   Zipcode Population  13556 non-null  int64         
 8   Date                13556 non-null  datetime64[ns]
 9   Zipcode Date        13556 non-null  object        
 10  Zipcode latitude    13556 non-null  float64       
 11  Zipcode longitude   13556 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 1.2+ MB


In [37]:
todaycase = df.loc[(df['Date']=="2021-01-04")]
todaycase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 290 to 13554
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Zipcode             47 non-null     int64         
 1   County              47 non-null     object        
 2   Statistic           47 non-null     object        
 3   Value               47 non-null     float64       
 4   Date used           47 non-null     object        
 5   Unit                47 non-null     object        
 6   Age adjusted        47 non-null     object        
 7   Zipcode Population  47 non-null     int64         
 8   Date                47 non-null     datetime64[ns]
 9   Zipcode Date        47 non-null     object        
 10  Zipcode latitude    47 non-null     float64       
 11  Zipcode longitude   47 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 4.8+ KB


In [41]:

m = folium.Map(location=[39.018541,-75.60966], zoom_start=7.5)
 
# I can add marker one by one on the map
for i in range(0,len(todaycase)):
   folium.Circle(
      location=[todaycase.iloc[i]['Zipcode latitude'], todaycase.iloc[i]['Zipcode longitude']],
      popup=todaycase.iloc[i]['Zipcode'],
      radius=todaycase.iloc[i]['Value']*9,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)
 
m


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13556 entries, 0 to 13555
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Zipcode             13556 non-null  int64         
 1   County              13556 non-null  object        
 2   Statistic           13556 non-null  object        
 3   Value               13556 non-null  float64       
 4   Date used           13556 non-null  object        
 5   Unit                13556 non-null  object        
 6   Age adjusted        13556 non-null  object        
 7   Zipcode Population  13556 non-null  int64         
 8   Date                13556 non-null  datetime64[ns]
 9   Zipcode Date        13556 non-null  object        
 10  Zipcode latitude    13556 non-null  float64       
 11  Zipcode longitude   13556 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 1.2+ MB
